<a href="https://www.armut.com">
    <img src="https://github.com/TarikKaanKoc/Armut-ARL_Recommender_System/blob/main/image.jpg?raw=True" width="8000" align="center">
</a>

<div class="alert alert-primary" style="margin-top: 20px">

    

<strong><h2>Bussines Problem</h2></strong>
---
Armut, Turkey's largest online service platform, brings together service providers and those who want to receive service. It provides easy access to services such as cleaning, modification and transportation with a few touches on your computer or smart phone. <p><strong>It is desired to create a product recommendation system with Association Rule Learning by using the data set containing the services and categories that the users have purchased.</strong></p>

---    
    
<strong><h2>Dataset Story</h2></strong>
<blockquote><p><strong>The data set consists of the services customers receive and the categories of these services. It contains the date and time information of each service received.</strong></p>
</blockquote>
    
<br></br>
<ul>
</ul>
<ul>

<li><strong>Total Features : 4</strong></li>
<li><strong>Total Row : 162.523</strong> </li>
<li><strong>CSV File Size : 5 MB</strong></li>

</ul>
</div>


 Sr. | Feature  | Description |
--- | --- | --- 
1 | UserId | Customer Id
2 | ServiceId | They are anonymized services belonging to each category. (Example: Upholstery washing service under cleaning) A ServiceId can be foundunder different categories and refers to different services under different categories. (Example: Furniture assembly with CategoryId 2 ServiceId 4 during service core cleaning with CategoryId 7 ServiceId 4) |
3 | CategoryId | They are anonymized categories. (Example: Cleaning, shipping, as follows)
4 | CreateDate | The date the service was purchased

---


<div class="inner_cell">
<div class="text_cell_render border-box-sizing rendered_html">
<p></p><div class="list-group" id="list-tab" role="tablist">
  <h3 class="list-group-item list-group-item-action active" data-toggle="list" role="tab" aria-controls="home" style = "border:2px solid #2ECC71;background-color:#2ECC71; color:white; font-family:Verdana;text-align: center; font-size:140%;font-weight: Bold;">Notebook Content</h3>
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#1" role="tab" aria-controls="profile" target="_self" style = "color:#2ECC71; font-family:Verdana;text-align: center; font-size:130%;font-weight: Bold;">Libraries<span class="badge badge-primary badge-pill">1</span></a>
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#2" role="tab" aria-controls="messages" target="_self" style = "color:#2ECC71; font-family:Verdana;text-align: center; font-size:130%;font-weight: Bold;">Load and Check Data<span class="badge badge-primary badge-pill">2</span></a>
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#2" role="tab" aria-controls="messages" target="_self" style = "color:#2ECC71; font-family:Verdana;text-align: center; font-size:130%;font-weight: Bold;">Data Preparation<span class="badge badge-primary badge-pill">3</span></a>
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#2" role="tab" aria-controls="messages" target="_self" style = "color:#2ECC71; font-family:Verdana;text-align: center; font-size:130%;font-weight: Bold;">Establishing and Suggesting Association Rules<span class="badge badge-primary badge-pill">4</span></a>
 
</div>
</div>
</div>

---

<a id = "2"></a>
<span style="color:#2ECC71;
             font-size:200%;
             font-family:Verdana;">
Libraries

In [2]:
from mlxtend.frequent_patterns import apriori, association_rules
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")
warnings.warn("this will not show")

<a id = "2"></a>
<span style="color:#2ECC71;
             font-size:200%;
             font-family:Verdana;">
Load and Check Data

In [3]:
df_ = pd.read_csv("../input/armut-data/armut_data.csv")

In [4]:
df = df_.copy()

In [5]:
def missing_values_analysis(df):
    na_columns_ = [col for col in df.columns if df[col].isnull().sum() > 0]
    n_miss = df[na_columns_].isnull().sum().sort_values(ascending=False)
    ratio_ = (df[na_columns_].isnull().sum() / df.shape[0] * 100).sort_values(ascending=False)
    missing_df = pd.concat([n_miss, np.round(ratio_, 2)], axis=1, keys=['Total Missing Values', 'Ratio'])
    missing_df = pd.DataFrame(missing_df).sort_values(by = "Ratio", ascending =False)
    return missing_df



def check_df(df, head=5, tail = 5):
    
    print(" SHAPE ".center(60,'~'))
    print('Rows: {}'.format(df.shape[0]))
    print('Columns: {}'.format(df.shape[1]))
    print(" TYPES ".center(60,'~'))
    print(df.dtypes)
    print(" HEAD ".center(60,'~'))
    print(df.head(head))
    print(' TAIL '.center(60,'~'))
    print(df.tail(tail))
    print(" HEAD ".center(60,'~'))
    print(missing_values_analysis(df))
    print(' DUPLICATED VALUES '.center(60,'~'))
    print(df.duplicated().sum())
    print(" QUANTILES ".center(60,'~'))
    print(df.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

check_df(df)

In [238]:
num_df = df_[["UserId","ServiceId","CategoryId"]]

In [239]:
def desc_stats(df):
    describe_ = df.describe().T
    describe_df = pd.DataFrame(index= df.columns, 
                           columns= describe_.columns,
                           data= describe_)
    
    f,ax = plt.subplots(figsize=(22,
                                 describe_df.shape[0]))
    sns.heatmap(describe_df,
                annot=True,
                cmap = "Blues",
                fmt= '.3f',
                ax=ax,
                linecolor='#C6D3E5',
                linewidths = 3,
                cbar = False,
                annot_kws={"size": 15})
    plt.xticks(size = 20)
    plt.yticks(size = 20,
               rotation = 0)
    plt.title("Descriptive Statistics", size = 25)
    plt.show()
    
desc_stats(num_df)

<a id = "2"></a>
<span style="color:#2ECC71;
             font-size:200%;
             font-family:Verdana;">
Data Preparation



<div class="alert alert-block alert-info">
        <b> 📌 ServiceID represents a different service for each CategoryID. We will combine ServiceID and CategoryID with "_" to create a new variable to represent these services. </b>
</div>
<br>

In [6]:
df['Service'] = [str(row[1]) + "_" + str(row[2])  for row in
                                   df.values]

df.head()

In [9]:
df[df["Service"] ==  "2_0"]



<div class="alert alert-block alert-info">
        <b> 📌 The data set consists of the date and time the services are received, there is no basket definition (invoice, etc.). In order to apply the Association Rule Learning, the definition of basket (invoice, etc.) must be created. Here, the definition of basket is the services that each customer receives on a monthly basis. For example; Basket consisting of 4_5, 48_5, 6_7, 47_7 services received by the customer with ID number 25446 in the 8th month of 2017; 17_5, 14_7 Services received in the 9th month of 2017 represent another basket. Baskets must be identified by a unique ID. To do this I will first create a new date variable that contains only the year and month. Then I will combine UserID and the new date variable with "_" and assign it to a new variable called ID.</b>
</div>
<br>

In [10]:
# Step - 1 
df['New_Date'] = pd.to_datetime(df['CreateDate'],format='%Y-%m').dt.to_period('M')
df.head()

In [11]:
# Step - 2
df['Basket_Id'] = [str(row[0]) + "_" + str(row[5])  for row in
                                   df.values]
df.head()

<a id = "2"></a>
<span style="color:#2ECC71;
             font-size:200%;
             font-family:Verdana;">
Establishing and Suggesting Association Rules

In [12]:
def create_invoice_product_df(df):
        return df.groupby(['Basket_Id', "Service"])['CategoryId'].sum().unstack().fillna(0). \
            applymap(lambda x: 1 if x > 0 else 0)
    
df_example = create_invoice_product_df(df)

In [18]:
df_example.head(20).style.background_gradient(axis=0, cmap='PuBu')

In [14]:
print("Total Number of Transactions -->", df_example.shape[0])
print("Total Number of Services -->", df.shape[1])

In [26]:
# creation of the apriori algorithm
apr_alg = df.groupby(['Basket_Id', 'Service'])["CategoryId"].count().unstack().fillna(0).applymap(lambda x: 1 if  x>0 else 0)
apr_alg = apriori(df_x, min_support=0.06, use_colnames=True, low_memory=True)
apr_alg.sort_values("support", ascending=False)

<div class="alert alert-block alert-info">
        <b> 📌When we look at the result above, we see the 5 most frequently requested services according to the support value. On top of that, the most frequently observed service is the 18_05 service. ( It is in the first place with 0.238121 Support value. )</b>
</div>
<br>

In [36]:
# Establishing association rules
apr_alg_rules = association_rules(apriori_alg,
                          metric="support",
                          min_threshold=0.01) # minimum support value

apr_alg_rules.sort_values("support", ascending=False)

In [39]:
# Filtering out those with lift value higher than 2
df_result= apr_alg_rules[apr_alg_rules["lift"]>2].sort_values("lift",ascending = False)
df_result

<div class="alert alert-block alert-info">
        <b> 📌 To suggest a service to a user who has received the 2_0 service in the last 1 month using the arl suggestion Function.

</b>
</div>
<br>

In [40]:
def arl_recommender(rules_df, service, rec_count=1):
    sorted_rules = rules_df.sort_values("lift", ascending=False)
    recommendation_list = []
    for i, serv in enumerate(sorted_rules["antecedents"]):
        for j in list(serv):
            if j == service:
                recommendation_list.append(list(sorted_rules.iloc[i]["consequents"])[0])

    return recommendation_list[0:rec_count]

arl_recommender(apr_alg_rules,"2_0")



<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:crimson;
           font-size:110%;
           font-family:Verdana;
           letter-spacing:0.5px">
🦅 If you like this notebook, please rate it 🥳

<br></br>
If you have any suggestions or questions, feel free to comment! 🐳🐳🐳

Good work! 🦾
</div>